<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Web/blob/main/141-Llamaindex01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LlamaIndex <> Langchain Integrations

이 데모 노트북은 LlamaIndex와 Langchain 간의 통합을 제공할 수 있는 방법을 보여줍니다. 다음 예제를 제공합니다.:
- Langchain 에이전트와 함께 호출 가능한 도구로 LlamaIndex 사용
- LlamaIndex를 메모리 모듈로 사용 이를 통해 Langchain 챗봇에 임의의 양의 대화 기록을 삽입할 수 있습니다!

In [ ]:
!pip install langchain langchain-core langchain-community python-dotenv
!pip install openai==0.28

In [ ]:
!pip install llama-index pypdf

In [ ]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Using LlamaIndex as a Callable Tool

In [ ]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [ ]:
!mkdir data

In [ ]:
import urllib.request

url = 'https://github.com/JSJeong-me/GPT-Web/raw/main/images/paul_graham_essay.txt'
filename = './data/paul_graham_essay.txt'

urllib.request.urlretrieve(url, filename)

# 파일 내용 확인 (처음 10줄 출력)
with open(filename, 'r', encoding='utf-8') as f:
    for _ in range(10):
        print(f.readline().rstrip())


In [ ]:
documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents=documents)

In [ ]:
tools = [
    Tool(
        name="LlamaIndex",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="useful for when you want to answer questions about the author. The input to this tool should be a complete english sentence.",
        return_direct=True,
    ),
]

In [ ]:
# set Logging to DEBUG for more detailed outputs
memory = ConversationBufferMemory(memory_key="chat_history")
llm = ChatOpenAI(temperature=0)
agent_executor = initialize_agent(
    tools, llm, agent="conversational-react-description", memory=memory
)

In [ ]:
agent_executor.run(input="hi, i am bob")

In [ ]:
agent_executor.run(input="what's my name?")

In [ ]:
agent_executor.run(input="who is the author of this document?")

In [ ]:
agent_executor.run(input="please summarize the document")

In [ ]:
agent_executor.run(input="please summarize the document in Korean")